In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.8.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Connect to your workspace

In [11]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [12]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


# Train a model and dump it for scoring later
The following part trains a simple model and dump it in a directory as a pickle file. 

In [ ]:
# Open terminal and move to the src folder and run to save the model
!python train-model-parameters.py --training_data diabetes.csv

# Create and Define Endpoint

In [13]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for diabetes model",
    auth_mode="key",
)

In [14]:
online_endpoint_name
endpoint

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpoint-08101919339814', 'description': 'Online endpoint for diabetes model', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/farbodtaymouri2/code/my-azure-ml-projects/model-deployment', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fe8fb939ba0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

### It is good to deploy locally
https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=python#deploy-the-model-locally

In [15]:
# ml_client.begin_create_or_update(endpoint).result()
ml_client.online_endpoints.begin_create_or_update(endpoint, local=False)

In [16]:
# Printing the list of online endpoints
eps_online = ml_client.online_endpoints.list()
for ep in eps_online:
    print(ep.name)

endpoint-08101919339814


# Configure deployment
## Creating scoring file

In [ ]:
#https://learn.microsoft.com/en-us/training/modules/deploy-model-managed-online-endpoint/4-eploy-custom-model-managed-online-endpoint

# See the /src/score.py in the folder

## How to test the scoring script for deployment locally from terminal
https://learn.microsoft.com/en-us/azure/machine-learning/how-to-inference-server-http?view=azureml-api-2

In [ ]:
# Run the followings from terminal

# Install the following package (you can create a new virtual env):
    # python -m pip install azureml-inference-server-http

# Open the terminal and run the following code which indicates the score.py and the directory of the model 
# see the score.py to understand the location of the model
    # azmlinfsrv --entry_script ./src/model/score.py --model_dir ./src/

# Open another terminal and run the following
    # curl --request POST "127.0.0.1:5001/score" --header 'Content-Type:application/json' --data @sample-data.json


## Create an environment using a base Docker image, 
you can define the Conda dependencies in a conda.yml file

In [10]:
%%writefile 'src/model/conda.yml'

name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Overwriting src/model/conda.yml


In [7]:
# Creating an environment
from azure.ai.ml.entities import Environment

env = Environment(
    image="mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest",   # Double check this as it might fail the creation of image!!!!!
    conda_file="./src/model/conda.yml",
    name="farbod-deployment-environment",
    version= '9',
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'farbod-deployment-environment', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d1d56/resourceGroups/mlcertificate1/providers/Microsoft.MachineLearningServices/workspaces/ft_ml2/environments/farbod-deployment-environment/versions/9', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/farbodtaymouri2/code/my-azure-ml-projects/model-deployment', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fe9134932e0>, 'serialize': <msrest.serialization.Serializer object at 0x7fe913493be0>, 'version': '9', 'latest_version': None, 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.7', 'scikit-learn', 'pandas', 'numpy', 'matplotlib', 'azu

In [54]:
# See whether the enviornment is avaialble
envs = ml_client.environments.list()
for my_env in envs:
    print(my_env.name)


farbod-deployment-environment
deployment-environment
DefaultNcdEnv-mlflow-ubuntu20-04-py38-cpu-inference
AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu
AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu
AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu
AzureML-ACPT-pytorch-1.11-py38-cuda11.5-gpu
AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu
AzureML-responsibleai-0.21-ubuntu20.04-py38-cpu
AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu
AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu
AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu
AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu
AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu
AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu
AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu
AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu
AzureML-Triton
AzureML-Designer-Score
AzureML-VowpalWabbi

## 

## Create the deployment

In [23]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment, CodeConfiguration
from azure.ai.ml.constants import AssetTypes

# Registering the model. Note that, during the deployment, the model will be accessed via  local path 'AZUREML_MODEL_DIR/model/model.pkl'
model = Model(path="./src/model/", description='my simple custom model')

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name= online_endpoint_name,
    model=model,
    # environment= env,
    # environment= "farbod-deployment-environment@latest",    # If you created your env already. see https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-environments-v2?view=azureml-api-2&tabs=python
    environment= 'AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest',
    code_configuration=CodeConfiguration(
        code="./src/model", scoring_script="score.py"
    ),
    instance_type="STANDARD_E2S_V3",    # VERY IMPORTANT Check the quota before running
    instance_count=1,
)




In [24]:
# ml_client.online_deployments.begin_create_or_update(blue_deployment).result()
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=False, vscode_debug= False
)

Check: endpoint endpoint-08101919339814 exists
Uploading model (0.0 MBs): 100%|██████████| 3871/3871 [00:00<00:00, 14404.54it/s]


Uploading model (0.0 MBs): 100%|██████████| 3871/3871 [00:00<00:00, 25873.02it/s]




......................................

In [26]:
# verify (local) deployment
ml_client.online_endpoints.get(name=online_endpoint_name, local=False)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08101919339814.australiaeast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-08101919339814.australiaeast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08101919339814', 'description': 'Online endpoint for diabetes model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d1d56/resourcegroups/mlcertificate1/providers/microsoft.machinelearningservices/workspaces/ft_ml2/onlineendpoints/endpoint-08101919339814', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d1d56/providers/Microsoft.MachineLearningServices/locations/australiaeast/mfeOperationsStatus/oe:44ba2fae-86c8-4414-b8fd-2b7466f5958e:af50b626-79fe-4a1b-b8c1-4fe47efe45a9?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d

In [25]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08101919339814.australiaeast.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-08101919339814.australiaeast.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08101919339814', 'description': 'Online endpoint for diabetes model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d1d56/resourcegroups/mlcertificate1/providers/microsoft.machinelearningservices/workspaces/ft_ml2/onlineendpoints/endpoint-08101919339814', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d1d56/providers/Microsoft.MachineLearningServices/locations/australiaeast/mfeOperationsStatus/oe:44ba2fae-86c8-4414-b8fd-2b7466f5958e:af50b626-79fe-4a1b-b8c1-4fe47efe45a9?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/2a21ade8-9d70-4d5a-a619-083b264d

## Test the deployment

In [27]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

print('response:', response)
if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

response: [0]
Not diabetic
